### Prepping Data Challenge: Pokemon Hierarchies (week 10)
The aim of this challenge is to group Pokémon into Evolution Groups. Why? So that when we select any Pokémon, we can see all the information about what they evolved from, what they will evolve to etc. 

How is this applicable to other hierarchical data? Well, imagine that a Pokémon that has a null in the Evolves From field is similar to an employee at Work Level 1 in an organisation. Their manager would then sit in the Evolves To field. Our Evolution Group in this instance would be the Team the employee belongs to and it would give us the ability to type in any employee name and return information about which team they belong to and the structure of that team. 

#### Requirement:
 1. Input the data
 2. Our Pokémon dataset actually contains too many Pokémon: 
    - We're only interested in Pokémon up to Generation III, which is up to (and including) number 386
    - This means we're also not interested in mega evolutions so we can filter Pokémon whose name start with "Mega"
 3. Some Pokémon have more than one Type. We aren't interested in Types for this challenge so remove this field and ensure we have one row per Pokémon 
 4. Now we want to bring in information about what our Pokémon evolve to 
    - Warning!  In our Evolution dataset, we still have Pokémon beyond Gen III. You'll need to filter these out too, from both the evolved from and evolved to fields 
 5. Bring in information about what a Pokémon evolves from 
    - Ensure that we have all 386 of our Pokémon, with nulls if they don't have a pre-evolved form or if they don't evolve
 6. Finally, for Pokémon that have 3 evolutions, we want to know what the First Evolution is in their Evolution Group 
 7. Some duplication may have occurred with all our joins, ensure no 2 rows are exactly the same 
 8. Create a calculation for our Evolution Group 
    - The Evolution Group will be named after the First Evolution e.g. in the above example, Bulbasaur is the name of the Evolution Group
 9. Output the data

### 1. Input the data

In [1]:
#import libraries
import pandas as pd
import numpy as np

In [2]:
with pd.ExcelFile('WK10-Pokemon.xlsx') as xlsx:
    pokemon = pd.read_excel(xlsx, 'Pokemon')
    evolution = pd.read_excel(xlsx, 'Evolution')

In [3]:
pokemon.head()

,#,Name,Type,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed
0,001,Bulbasaur,GRASS,318,45,49,49,65,65,45
1,001,Bulbasaur,POISON,318,45,49,49,65,65,45
2,002,Ivysaur,GRASS,405,60,62,63,80,80,60
3,002,Ivysaur,POISON,405,60,62,63,80,80,60
4,003,Venusaur,GRASS,525,80,82,83,100,100,80


In [4]:
evolution.head()

,Evolving from,Evolving to,Level,Condition,Evolution Type
0,Bulbasaur,Ivysaur,16.0,NaN,Level
1,Ivysaur,Venusaur,32.0,NaN,Level
2,Charmander,Charmeleon,16.0,NaN,Level
3,Charmeleon,Charizard,36.0,NaN,Level
4,Squirtle,Wartortle,16.0,NaN,Level


###  2. Our Pokémon dataset actually contains too many Pokémon:

In [5]:
#We're only interested in Pokémon up to Generation III, which is up to (and including) number 386
#This means we're also not interested in mega evolutions so we can filter Pokémon whose name start with "Mega"
pokemon['#']=pokemon['#'].astype('float')
pokemon=pokemon.loc[pokemon['#']<=386,]
pokemon=pokemon[~(pokemon['Name'].str.startswith('Mega'))]

### 3. Some Pokémon have more than one Type. We aren't interested in Types for this challenge so remove this field and ensure we have one row per Pokémon

In [6]:
pokemon.drop('Type',axis=1,inplace=True)
pokemon.drop_duplicates(inplace=True)

### 4. Now we want to bring in information about what our Pokémon evolve to 

In [7]:
#Warning!  In our Evolution dataset, we still have Pokémon beyond Gen III. 
#You'll need to filter these out too, from both the 'evolved from' and 'evolved to' fields
names_list = pokemon['Name']
evolve = evolution[evolution['Evolving from'].isin(names_list)]
evolve = evolution[evolution['Evolving to'].isin(names_list)]

### 5. Bring in information about what a Pokémon evolves from

In [8]:
evolve_dict = dict(zip(evolution['Evolving to'], evolution['Evolving from']))

df = pokemon.merge(evolution, left_on='Name', right_on='Evolving from', how='left')
df['Evolving from'] = [evolve_dict[k] if k in evolve_dict.keys() else np.NaN for k in df['Name']]

### 6 & 8. Finally, for Pokémon that have 3 evolutions, we want to know what the First Evolution is in their Evolution Group 

In [9]:
def get_evolve_group(names):
    if names not in evolve_dict.keys() or names == evolve_dict[names]: 
        return names
    else:
        return get_evolve_group(evolve_dict[names])

In [10]:
df['Evolution Group'] = df['Name'].apply(get_evolve_group)

In [11]:
df['First Evolution'] = [np.nan if (n==g) | (g==f) else g 
                         for n,g,f in zip(df['Name'], df['Evolution Group'], df['Evolving from'])]

### 7. Some duplication may have occurred with all our joins, ensure no 2 rows are exactly the same

In [12]:
df.drop_duplicates(inplace=True)

### 12. Output the data 

In [13]:
df=df[['Evolution Group','#','Name','Total','HP','Attack','Defense',
            'Special Attack','Special Defense','Speed','Evolving from',
            'Evolving to','Level','Condition','Evolution Type',
            'First Evolution']]

In [15]:
df.to_csv('WK10-Pokemon Output.csv', index=False)

In [16]:
df.head()

,Evolution Group,#,Name,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed,Evolving from,Evolving to,Level,Condition,Evolution Type,First Evolution
0,Bulbasaur,1.0,Bulbasaur,318,45,49,49,65,65,45,NaN,Ivysaur,16.0,NaN,Level,NaN
1,Bulbasaur,2.0,Ivysaur,405,60,62,63,80,80,60,Bulbasaur,Venusaur,32.0,NaN,Level,NaN
2,Bulbasaur,3.0,Venusaur,525,80,82,83,100,100,80,Ivysaur,NaN,NaN,NaN,NaN,Bulbasaur
3,Charmander,4.0,Charmander,309,39,52,43,60,50,65,NaN,Charmeleon,16.0,NaN,Level,NaN
4,Charmander,5.0,Charmeleon,405,58,64,58,80,65,80,Charmander,Charizard,36.0,NaN,Level,NaN
